In [ ]:
import scipy
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
J = 0.0005 # kg*m^2, rotor inertia
T_1 = 1/7.66 # gear ratio of link 1
T_2 = 1/8.846 # gear ratio link 2
b = 0.1

T = T_2

# rotor vibrates at this frequency when hit
# equivalent to a spring
omega_resonant_hz = 30.0
omega_resonant_rad_s = 2 * np.pi * 30.0
gearbox_stiffness_before_reduction = omega_resonant_rad_s**2 * J

k_s = gearbox_stiffness_before_reduction

print gearbox_stiffness_before_reduction
chain_stiffness = gearbox_stiffness_before_reduction * (1/T_2)**2
print chain_stiffness

## Linear Systems Analysis using Bode Plot
Frequency domain analysis

In [ ]:
# make bode plot
# input is "torque desired, i.e. tau = G/T * i", current that you would need to get
# a specific torque out, G is the torque motor constant
system = scipy.signal.lti(np.array([k_s]), [J, b, k_s])
f = np.logspace(0,5)
w = 2 * np.pi * f
w, mag, phase = scipy.signal.bode(system, w)

plt.semilogx(f, mag)
plt.show()

plt.semilogx(f, phase)
plt.show()

## Time Domain System setup
Setup the system in the time domain. State variables $[\theta, \dot{\theta}]$. Control variables are
$u = [\tau, x_b]$ where $\tau = \frac{G}{T} i$, i.e. we think of commading desired output torque.

In [ ]:
A = np.array([[0, 1], [-k_s/J, -b/J]])
B = np.array([[0,0], [T/J, -k_s/(T*J)]])
C = np.array([k_s/T, 0])
D = np.array([0, k_s/T])



# single dimensional input
if True:
    B = np.array([[0], [T/J]])
    D = 0
    
print "A\n", A
print "B\n", B
print "C\n", C
print "D\n", D


sys_2 = scipy.signal.lti(A, B, C, D)
sys_2_tf = sys_2.to_tf()
print sys_2_tf

tf = scipy.signal.TransferFunction(sys_2)

# make bode plot
w, mag, phase = scipy.signal.bode(sys_2, w)

plt.semilogx(f, mag)
plt.show()

plt.semilogx(f, phase)
plt.show()

## Simulate the response to a step input

Damping turns out to be very important to ensure that the system doesn't oscillate endlessly

In [ ]:
output_torque = 100.0
T_vec = np.linspace(0,0.2,100)
T_vec, yout = scipy.signal.step(system, T=T_vec)
plt.plot(T_vec, yout)